In [10]:
import sys
import os
import mmh3
from sklearn.feature_extraction.text import TfidfVectorizer


In [11]:
%load_ext autoreload
%autoreload 2
!python -m pip install -r requirements.txt
from utils.initialization import *


# med mrjob
# names, party and twitter id
from Data.twitter_ids import twitter_ids
data = pd.DataFrame(columns=['name', "party", 'twitter_id'])
i = 0
for party in twitter_ids:
    for person in twitter_ids[party]:
        data.loc[i, :] = [person, party, twitter_ids[party][person]]
        i += 1

# tweets
filename = "Data/cleaned_data.csv"
if not os.path.exists(filename):
    os.system(f"python utils/clean_data_mrjob.py Data/tweets > Data/tmp_cleaned_data.txt")
    data_ = pd.DataFrame(columns=["name", "tweets"])
    with open("Data/tmp_cleaned_data.txt", "rb") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            line = eval(line.decode())
            data_.loc[i,"name"] = list(line.keys())[0]
            data_.loc[i, "tweets"] = list(line.values())[0]
    data_.to_csv(filename, index = False)

data_ = pd.read_csv(filename)
data = data.merge(data_)
data.tweets = [eval(t) for t in data.tweets]
data["tokens"] = [[w for w in word_tokenize(" ".join(data["tweets"][i])) if w.isalnum()] for i in range(len(data))]
import simplemma #use simplemma instead of nltk.WordNetLemmatizer()          ### vi har addet denne linje
data["tokens"] = [[simplemma.lemmatize(w, lang='da') for w in data["tokens"][i]] for i in range(len(data))] ### vi har addet denne linje

# only include unique words
data['tokens'] = data['tokens'].apply(lambda x: list(set(x)))

data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


You should consider upgrading via the 'c:\Users\bayka\Anaconda3\envs\katrine_personal_env\python.exe -m pip install --upgrade pip' command.


,name,party,twitter_id,tweets,tokens
0,magnus_heunicke,socialdemokratiet,22695562,[afsætter året styrke hjælpen børn pårørende a...,"[indkalde, sundhed, sekventeret, løsning, retn..."
1,nicolai_wammen,socialdemokratiet,2803948786,[dage siden sagde nyt ejendomsvurderingssystem...,"[indkalde, sundhed, dkp, løsning, retning, vrø..."
2,mattias_tesfaye,socialdemokratiet,546254893,[this is literally the same logic many th c am...,"[indkalde, sundhed, vacation, løsning, isolere..."
3,jakob_ellemann,venstre,155584627,[tide få fleksibel genåbning vores børn ældre ...,"[indkalde, sundhed, løsning, isolere, retning,..."
4,soren_gade,venstre,975064362359623680,[kære marianne synes burde læse lovforslaget i...,"[tusind, øje, mindsteløn, løsning, isolere, ve..."
5,sophie_lohde,venstre,44611200,[flertallet veto dermed røde partier stort set...,"[indkalde, sundhed, løsning, modstrid, retning..."
6,lars_lokke,moderaterne,26201346,[mon ikke sjov form argumentation mangler lidt...,"[indkalde, forsætlig, dkp, løsning, retning, i..."
7,jacob_mark,sf,2373406198,[slår fast syvtommersøm kom så godt igennem fo...,"[indkalde, sundhed, mobning, dkp, løsning, ret..."
8,pia_dyhr,sf,65025162,"[stemmer nok selvom synes gør godt klaus, brug...","[mobning, sundhed, dkp, løsning, decidere, vrø..."
9,kirsten_andersen,sf,235646319,[arbejde få medarbejdere ser virkeligheden sun...,"[indkalde, sundhed, ditto, løsning, retning, i..."


In [12]:
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 
    
def minhash(shingles_list, seed):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - seed (int): seed for listhash function
    Return: minhash of given shingles
    """
    minhash_value = None
    for aShingle in shingles_list:
        hashcode = listhash([aShingle], seed)
        if minhash_value == None or hashcode < minhash_value:
            minhash_value = hashcode
    return minhash_value

def minhash2(shingles_list, k):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - k (int): seed for listhash function
    Return: sequence of k minhashes
    """
    all_minhash = []
    for i in range(k):
        all_minhash.append(minhash(shingles_list, i))
    return all_minhash

In [27]:
k=200


# def signature(dict_docs, q = q, num_hashes = k):
def signature(dict_docs, num_hashes = k, on_what = None):
    """
    Input:
        - dict_docs (dict of str:str): dictionary of {title:document}
        - q (int)
        - num_hashes (int)
    Return: dictionary consisting of document id’s as keys and signatures as values
    """
    dict_signatures = {}
    total_texts = len(list(dict_docs.keys()))
    counter = 1
    for key,text in dict_docs.items():
        print(f'{counter}/{total_texts} - {key} - Processing...')
        # doc_shingles = shingle(text, q)
        # doc_shingles = data.tokens[counter-1] ### TODO lave denne om til at bruge keyword on
        doc_shingles = data[on_what][counter-1] ### TODO lave denne om til at bruge keyword on

        # doc_shingles = mini_df.TFIDF_Words[counter-1]
        minhash_values = minhash2(doc_shingles, num_hashes)
        dict_signatures[key] = minhash_values
        counter += 1
    return dict_signatures



In [28]:
def jaccard(name1, name2, signature_dict):
    """
    Input:
        - name1 (str): key of the first document S
        - name2 (str): key of the second document T
        - signatures_dict (dict of str:list): dictionary of signatures
    Return: Jaccard similarity between S and T
    """
    signatures_doc1 = np.array(signature_dict[name1])
    signatures_doc2 = np.array(signature_dict[name2])
    # return np.sum(signatures_doc1 == signatures_doc2) #/ len(signatures_doc1)
    return len(np.intersect1d(signatures_doc1, signatures_doc2))/len(np.union1d(signatures_doc1, signatures_doc2))#, np.setdiff1d(signatures_doc1, signatures_doc2)

def similar(signatures_dict, jaccard_threshold=0.05):
    """
    Input:
        - signatures_dict (dict of str:list): dictionary of signatures
        - jaccard_threshold (float): lower bound for Jaccard similarity score to consider
            two documents as similar
    Return: dictionary of similar items
    """
    list_keys = list(signatures_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
            if similarity_score >= jaccard_threshold:
                similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

In [29]:
# for every person, return the 3 people most similar to him/her
def most_similar_persons(similar_items, num_similar_persons=3):
    """
    Input:
        - similar_items (dict of tuple:str): dictionary of similar items
        - num_similar_persons (int): number of similar persons to return
    Return: dictionary of most similar persons
    """
    most_similar_persons = {}
    for key,value in similar_items.items():
        if key[0] not in most_similar_persons:
            most_similar_persons[key[0]] = [(key[1], value)]
        else:
            most_similar_persons[key[0]].append((key[1], value))
        if key[1] not in most_similar_persons:
            most_similar_persons[key[1]] = [(key[0], value)]
        else:
            most_similar_persons[key[1]].append((key[0], value))
    for key,value in most_similar_persons.items():
        most_similar_persons[key] = sorted(value, key=lambda x: x[1], reverse=True)[:num_similar_persons]
    return most_similar_persons

# most_similar_persons_res = most_similar_persons(found_similar_items)

In [30]:


def create_sim_table(df, level, on):
    if on == 'TFIDF_Words':
        vectorizer = TfidfVectorizer(max_df=0.3, max_features=700, stop_words='english')
        vectors = vectorizer.fit_transform(df.tokens.apply(lambda x: " ".join(x)))
        dict_of_tokens={i[1]:i[0] for i in vectorizer.vocabulary_.items()}
        tfidf_vectors = []  # all deoc vectors by tfidf
        for row in vectors:
            tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})
        df['TFIDF_Words'] = [list(tfidf_vectors[i].keys()) for i in range(len(df))]
    elif on == 'tokens':
        pass
    
    dict_docs = {i:j for i,j in zip(df[level],df[on])}
    signature_dict = signature(dict_docs, num_hashes = 200, on_what = on)
    found_similar_items = similar(signature_dict)
    most_similar_items = sorted(found_similar_items.items(), key=lambda x: x[1], reverse=True)
    most_similar_persons_res = most_similar_persons(found_similar_items)

    # convert most_similar_persons_res to a datamframe for easier visualization
    most_similar_persons_df = pd.DataFrame.from_dict(most_similar_persons_res, orient='index')
    if level == 'name':
        most_similar_persons_df.columns = ['Most similar person 1', 'Most similar person 2', 'Most similar person 3']
    elif level == 'party':
        most_similar_persons_df.columns = ['Most similar party 1', 'Most similar party 2', 'Most similar party 3']
    return most_similar_persons_df



In [31]:
# for each politician 
create_sim_table(data, 'name','tokens')

1/33 - magnus_heunicke - Processing...
2/33 - nicolai_wammen - Processing...
3/33 - mattias_tesfaye - Processing...
4/33 - jakob_ellemann - Processing...
5/33 - soren_gade - Processing...
6/33 - sophie_lohde - Processing...
7/33 - lars_lokke - Processing...
8/33 - jacob_mark - Processing...
9/33 - pia_dyhr - Processing...
10/33 - kirsten_andersen - Processing...
11/33 - dennis_flydtkjær - Processing...
12/33 - peter_skaarup - Processing...
13/33 - soren_espersen - Processing...
14/33 - alex_vanopslagh - Processing...
15/33 - ole_olesen - Processing...
16/33 - soren_pape - Processing...
17/33 - mette_abildgaard - Processing...
18/33 - rasmus_jarlov - Processing...
19/33 - pelle_dragsted - Processing...
20/33 - mai_villadsen - Processing...
21/33 - rosa_lund - Processing...
22/33 - martin_lidegaard - Processing...
23/33 - samira_nawa - Processing...
24/33 - katrine_robsoe - Processing...
25/33 - pernille_vermund - Processing...
26/33 - lars_mathiesen - Processing...
27/33 - kim_andersen 

,Most similar person 1,Most similar person 2,Most similar person 3
magnus_heunicke,"(kirsten_andersen, 0.1396011396011396)","(alex_vanopslagh, 0.1267605633802817)","(martin_lidegaard, 0.1267605633802817)"
nicolai_wammen,"(dennis_flydtkjær, 0.14613180515759314)","(jakob_ellemann, 0.1267605633802817)","(pernille_vermund, 0.1267605633802817)"
mattias_tesfaye,"(alex_vanopslagh, 0.14613180515759314)","(rosa_lund, 0.13636363636363635)","(peter_skaarup, 0.13314447592067988)"
jakob_ellemann,"(soren_pape, 0.2084592145015106)","(sophie_lohde, 0.19402985074626866)","(lars_lokke, 0.17994100294985252)"
sophie_lohde,"(jakob_ellemann, 0.19402985074626866)","(soren_pape, 0.18694362017804153)","(mette_abildgaard, 0.17302052785923755)"
lars_lokke,"(rosa_lund, 0.19047619047619047)","(jakob_ellemann, 0.17994100294985252)","(alex_vanopslagh, 0.17647058823529413)"
jacob_mark,"(kirsten_andersen, 0.17647058823529413)","(rasmus_jarlov, 0.1661807580174927)","(jakob_ellemann, 0.15942028985507245)"
pia_dyhr,"(mette_abildgaard, 0.17994100294985252)","(jakob_ellemann, 0.17647058823529413)","(kirsten_andersen, 0.17647058823529413)"
kirsten_andersen,"(jacob_mark, 0.17647058823529413)","(pia_dyhr, 0.17647058823529413)","(jakob_ellemann, 0.16279069767441862)"
dennis_flydtkjær,"(jakob_ellemann, 0.15273775216138327)","(ole_olesen, 0.14942528735632185)","(pernille_vermund, 0.14942528735632185)"


In [32]:
# for each party
create_sim_table(data, 'party','TFIDF_Words')

1/12 - socialdemokratiet - Processing...
2/12 - venstre - Processing...
3/12 - moderaterne - Processing...
4/12 - sf - Processing...
5/12 - danmarksdemokraterne - Processing...
6/12 - liberal_alliance - Processing...
7/12 - konservative - Processing...
8/12 - enhedslisten - Processing...
9/12 - radikale - Processing...
10/12 - nye_borgerlige - Processing...
11/12 - alternativet - Processing...
12/12 - dansk_folkeparti - Processing...


,Most similar party 1,Most similar party 2,Most similar party 3
socialdemokratiet,"(liberal_alliance, 0.13636363636363635)","(nye_borgerlige, 0.12359550561797752)","(dansk_folkeparti, 0.10803324099722991)"
venstre,"(moderaterne, 0.08401084010840108)","(alternativet, 0.07526881720430108)","(liberal_alliance, 0.06951871657754011)"
moderaterne,"(dansk_folkeparti, 0.1111111111111111)","(enhedslisten, 0.08991825613079019)","(venstre, 0.08401084010840108)"
sf,"(konservative, 0.10497237569060773)","(moderaterne, 0.08401084010840108)","(socialdemokratiet, 0.07238605898123325)"
liberal_alliance,"(dansk_folkeparti, 0.15606936416184972)","(socialdemokratiet, 0.13636363636363635)","(enhedslisten, 0.09289617486338798)"
konservative,"(radikale, 0.1111111111111111)","(sf, 0.10497237569060773)","(liberal_alliance, 0.08695652173913043)"
enhedslisten,"(nye_borgerlige, 0.1396011396011396)","(socialdemokratiet, 0.10497237569060773)","(dansk_folkeparti, 0.0989010989010989)"
radikale,"(konservative, 0.1111111111111111)","(enhedslisten, 0.0958904109589041)","(nye_borgerlige, 0.07238605898123325)"
nye_borgerlige,"(enhedslisten, 0.1396011396011396)","(socialdemokratiet, 0.12359550561797752)","(dansk_folkeparti, 0.1111111111111111)"
alternativet,"(dansk_folkeparti, 0.08108108108108109)","(venstre, 0.07526881720430108)","(nye_borgerlige, 0.07238605898123325)"


In [33]:
create_sim_table(data, 'party','tokens')

1/12 - socialdemokratiet - Processing...
2/12 - venstre - Processing...
3/12 - moderaterne - Processing...
4/12 - sf - Processing...
5/12 - danmarksdemokraterne - Processing...
6/12 - liberal_alliance - Processing...
7/12 - konservative - Processing...
8/12 - enhedslisten - Processing...
9/12 - radikale - Processing...
10/12 - nye_borgerlige - Processing...
11/12 - alternativet - Processing...
12/12 - dansk_folkeparti - Processing...


,Most similar party 1,Most similar party 2,Most similar party 3
socialdemokratiet,"(nye_borgerlige, 0.1396011396011396)","(sf, 0.12359550561797752)","(enhedslisten, 0.12359550561797752)"
venstre,"(alternativet, 0.14613180515759314)","(sf, 0.1267605633802817)","(radikale, 0.11731843575418995)"
moderaterne,"(dansk_folkeparti, 0.13314447592067988)","(sf, 0.12994350282485875)","(liberal_alliance, 0.1267605633802817)"
sf,"(liberal_alliance, 0.19402985074626866)","(konservative, 0.17994100294985252)","(radikale, 0.17647058823529413)"
liberal_alliance,"(sf, 0.19402985074626866)","(radikale, 0.1695906432748538)","(nye_borgerlige, 0.16279069767441862)"
konservative,"(sf, 0.17994100294985252)","(dansk_folkeparti, 0.15606936416184972)","(nye_borgerlige, 0.14942528735632185)"
enhedslisten,"(nye_borgerlige, 0.17647058823529413)","(sf, 0.15942028985507245)","(liberal_alliance, 0.14942528735632185)"
radikale,"(sf, 0.17647058823529413)","(nye_borgerlige, 0.17647058823529413)","(liberal_alliance, 0.1695906432748538)"
nye_borgerlige,"(enhedslisten, 0.17647058823529413)","(radikale, 0.17647058823529413)","(sf, 0.16279069767441862)"
alternativet,"(sf, 0.15273775216138327)","(venstre, 0.14613180515759314)","(liberal_alliance, 0.1396011396011396)"


# Recommendation system 

# Make a function that takes one person, and recommends the top 3 politicians and the top 3 parties they agree with. 


In [ ]:
def recommend(name, most_similar_persons_df):
    # get the most similar persons to the input person

    return 